In [9]:
from tqdm.auto import tqdm
from call_model import call_chat_once
import asyncio
import httpx
from uuid import uuid4
import json
import datasets
from typing import Optional
import os
import csv
import nest_asyncio
nest_asyncio.apply()

from huggingface_hub import login
from datetime import datetime, timezone
import time
from app.config import configs

login(token=configs.HF_TOKEN)

In [10]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train") # or load from data/test.csv
print(eval_dataset)

Dataset({
    features: ['context', 'question', 'answer', 'source_doc', 'standalone_score', 'standalone_eval', 'relatedness_score', 'relatedness_eval', 'relevance_score', 'relevance_eval'],
    num_rows: 65
})


In [11]:
OUTPUT_FILE = "data/rag_results_default_LLMListwiseRerank_gpt4omini__k_retriever=15_tune_prompt_1.csv"

def load_existing_results(file_path):
    dataset = []
    existing_questions = set()
    if os.path.exists(file_path):
        with open(file_path, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing_questions.add(row["user_input"])
                # parse retrieved_contexts lại từ JSON
                row["retrieved_contexts"] = json.loads(row["retrieved_contexts"])
                dataset.append(row)
    return existing_questions, dataset


def append_result_to_csv(file_path, row, fieldnames):
    file_exists = os.path.exists(file_path)
    with open(file_path, "a", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    verbose: Optional[bool] = False
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    fieldnames = ["user_input", "retrieved_contexts", "response", "reference"]

    processed_questions, dataset = load_existing_results(OUTPUT_FILE)
    print(f"Đã có {len(processed_questions)} câu hỏi xử lý trước đó, sẽ bỏ qua chúng.")


    for i, example in enumerate(tqdm(eval_dataset)):
        question = example["question"]

        if question in processed_questions:
            continue
        
        payload = {
            "question": question,
            "session_id": str(uuid4()),
            "chat_history": []
        }

        try:
            relevant_docs, answer = asyncio.run(call_chat_once(payload))
        except Exception as e:
            print(f"Error from question: '{question}': {e}")

        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')

        row = {
            "user_input": question,
            "retrieved_contexts": json.dumps(
                [doc['kwargs'].get('page_content') for doc in relevant_docs],
                ensure_ascii=False
            ),
            "response": answer,
            "reference": example['answer']
        }
        append_result_to_csv(OUTPUT_FILE, row, fieldnames)
        dataset.append({
            "user_input": question,
            "retrieved_contexts": json.loads(row["retrieved_contexts"]),
            "response": answer,
            "reference": example['answer']
        })
    return dataset

In [12]:
data = run_rag_tests(eval_dataset=eval_dataset)


Đã có 0 câu hỏi xử lý trước đó, sẽ bỏ qua chúng.


  2%|▏         | 1/65 [00:05<06:03,  5.68s/it]

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What architecture is the `tokenizers-linux-x64-musl` binary designed for?\n', 'type': 'human', 'id': 'a5e27585-46de-4bff-b834-58731f291b7f'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_9aI5i08ySElMxxp779nxjW2Z', 'function': {'arguments': '{"query":"tokenizers-linux-x64-musl architecture"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab'}, 'type': 'ai', 'id': 'run--ad697d03-4ab9-4ae0-a346-522ccf9c65ff-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'tokenizers-linux-x64-musl architecture'}, 'id': 'call_9aI5i08ySElMxxp779nxjW2Z', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 289, 'o

2025-08-24 21:09:32,230 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
  3%|▎         | 2/65 [00:12<06:22,  6.06s/it]

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the purpose of the BLIP-Diffusion model?\n', 'type': 'human', 'id': '6c3d127d-36fc-475b-a618-681574a76d09'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_FYcJuk6N7Lefr505uC8su2dG', 'function': {'arguments': '{"query":"BLIP-Diffusion model purpose"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab'}, 'type': 'ai', 'id': 'run--781c372e-caa2-4c47-a14f-2b45db869ec3-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'BLIP-Diffusion model purpose'}, 'id': 'call_FYcJuk6N7Lefr505uC8su2dG', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 285, 'output_tokens': 22, 'total_tokens': 307, 'inpu

...

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the name of the open-source library created by Hugging Face to simplify Transformer acceleration?\n', 'type': 'human', 'id': '78b3cb71-a23a-493e-a4b0-3db2896ebcd8'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_Qrs1r1Cthz8Yba97kPBjGOV5', 'function': {'arguments': '{"query":"open-source library created by Hugging Face to simplify Transformer acceleration"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab'}, 'type': 'ai', 'id': 'run--334cbdc8-1385-4441-b6bf-3317d3a32669-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'open-source library created by Hugging Face to simplify Transformer acce

2025-08-24 21:14:54,068 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
 98%|█████████▊| 64/65 [05:33<00:04,  4.66s/it]

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What parameter is used to ensure that elements in a row have the same height in Gradio?\n', 'type': 'human', 'id': '33872273-597b-471d-af11-986ba57dc79a'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_N2RHRMLIu0FkrbQ1A9kfVCIN', 'function': {'arguments': '{"query":"Gradio same height elements in a row parameter"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab'}, 'type': 'ai', 'id': 'run--0e6e243c-a0b0-43f0-aaee-11de46ca89c6-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'Gradio same height elements in a row parameter'}, 'id': 'call_N2RHRMLIu0FkrbQ1A9kfVCIN', 'type': 'tool_call'}], 'usage_metad

2025-08-24 21:14:58,002 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
100%|██████████| 65/65 [05:37<00:00,  5.20s/it]

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the command to install the latest version of Optimum with OpenVINO support?\n', 'type': 'human', 'id': 'a770e584-0549-4827-8bcc-ab68e4fd05d3'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_pJNGMkCjfYt4Xsr2oWUpxeVI', 'function': {'arguments': '{"query":"install latest version of Optimum with OpenVINO support"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab'}, 'type': 'ai', 'id': 'run--9b0379ba-a86f-401d-8106-8f3cec719c66-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'install latest version of Optimum with OpenVINO support'}, 'id': 'call_pJNGMkCjfYt4Xsr2oWUpxeVI', 'type': 'tool_call'}]

In [13]:
!pip install ragas

In [14]:
from ragas import EvaluationDataset
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    FactualCorrectness
)
evaluation_dataset = EvaluationDataset.from_list(data)

# Khởi tạo các metric
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]
# metrics = [FactualCorrectness()]

In [15]:
print(len(evaluation_dataset))

65


In [16]:
from app.config import configs

import os
os.environ['OPENAI_API_KEY'] = configs.OPENAI_API_KEY

result = evaluate(
    evaluation_dataset,
    metrics=metrics,
)

print(result) 

Evaluating:   0%|          | 0/260 [00:00<?, ?it/s]2025-08-24 21:15:02,545 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-24 21:15:02,563 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-24 21:15:02,575 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-24 21:15:02,582 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-24 21:15:03,072 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-24 21:15:03,711 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-24 21:15:03,943 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-24 21:15:03,944 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-24 21:15:03,945 - INFO - HTTP Request: POST https

{'faithfulness': 0.8863, 'answer_relevancy': 0.9526, 'context_precision': 0.8833, 'context_recall': 0.8692}
